In [2]:
import torch
import torch.nn as nn
import math
from torch.nn import functional as F

In [3]:
qlen = 16
klen = 16
device = "cpu"

In [4]:
context_position = torch.arange(qlen, dtype=torch.long, device=device)[:, None]
print(context_position.shape)
context_position

torch.Size([16, 1])


tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10],
        [11],
        [12],
        [13],
        [14],
        [15]])

In [5]:
memory_position = torch.arange(klen, dtype=torch.long, device=device)[None, :]
print(memory_position.shape)
memory_position

torch.Size([1, 16])


tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]])

In [6]:
relative_position = memory_position - context_position
print(relative_position.shape)
relative_position

torch.Size([16, 16])


tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15],
        [ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
          13,  14],
        [ -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
          12,  13],
        [ -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
          11,  12],
        [ -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
          10,  11],
        [ -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,
           9,  10],
        [ -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,
           8,   9],
        [ -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,
           7,   8],
        [ -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,
           6,   7],
        [ -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,
           5,   6],
        [-

## Relative position bucket

In [7]:
bidirectional = True
num_buckets = 4
num_heads = 2
max_distance = 8

In [8]:
ret = 0
n = -relative_position
if bidirectional:
    # num_buckets //= 2
    # ret += (n < 0).to(torch.long) * num_buckets
    n = torch.abs(n)
else:
    n = torch.max(n, torch.zeros_like(n))

n

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
        [ 2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
        [ 3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [ 4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [ 5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
        [ 6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [ 7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7],
        [ 9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6],
        [10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5],
        [11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4],
        [12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3],
        [13, 12, 11, 10,  9,  8,  7,  

In [9]:
base = (max_distance)**(1/(num_buckets - 1))
n_buckets = torch.max(torch.log((n.float()))/math.log(base), torch.zeros_like(n)).to(torch.long)
n_buckets

tensor([[0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3],
        [0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3],
        [1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3],
        [1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3],
        [2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
        [2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3],
        [2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3],
        [2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3],
        [3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2],
        [3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2],
        [3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2],
        [3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2],
        [3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1],
        [3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1],
        [3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0]])

In [10]:
n_buckets = torch.min(n_buckets, torch.ones_like(n_buckets) * (num_buckets - 1))
n_buckets

tensor([[0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3],
        [0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3],
        [1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3],
        [1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3],
        [2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
        [2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3],
        [2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3],
        [2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3],
        [3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2],
        [3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2],
        [3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2],
        [3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2],
        [3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1, 1],
        [3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0, 1],
        [3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 0, 0]])

In [11]:
embeddings = nn.Embedding(num_embeddings=num_buckets, embedding_dim=num_heads)
embedded = embeddings(n_buckets)
embedded.shape

torch.Size([16, 16, 2])

In [12]:
embedded = embedded.permute([2, 0, 1])
embedded[0]

tensor([[-0.3238, -0.3238,  0.1103,  0.1103, -0.5091, -0.5091, -0.5091, -0.5091,
          1.9246,  1.9246,  1.9246,  1.9246,  1.9246,  1.9246,  1.9246,  1.9246],
        [-0.3238, -0.3238, -0.3238,  0.1103,  0.1103, -0.5091, -0.5091, -0.5091,
         -0.5091,  1.9246,  1.9246,  1.9246,  1.9246,  1.9246,  1.9246,  1.9246],
        [ 0.1103, -0.3238, -0.3238, -0.3238,  0.1103,  0.1103, -0.5091, -0.5091,
         -0.5091, -0.5091,  1.9246,  1.9246,  1.9246,  1.9246,  1.9246,  1.9246],
        [ 0.1103,  0.1103, -0.3238, -0.3238, -0.3238,  0.1103,  0.1103, -0.5091,
         -0.5091, -0.5091, -0.5091,  1.9246,  1.9246,  1.9246,  1.9246,  1.9246],
        [-0.5091,  0.1103,  0.1103, -0.3238, -0.3238, -0.3238,  0.1103,  0.1103,
         -0.5091, -0.5091, -0.5091, -0.5091,  1.9246,  1.9246,  1.9246,  1.9246],
        [-0.5091, -0.5091,  0.1103,  0.1103, -0.3238, -0.3238, -0.3238,  0.1103,
          0.1103, -0.5091, -0.5091, -0.5091, -0.5091,  1.9246,  1.9246,  1.9246],
        [-0.5091, -0.5

In [13]:
torch.tril(torch.ones((10, 10)))

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])